In [5]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import sys
from loguru import logger

logger.remove()
logger.add(sys.stderr, level="INFO")
data_path = Path("../data")
# data_path = Path('data')
config = pd.read_excel(data_path / "config.xlsx", sheet_name="layers")
config.tail(3)

,Name,shape_type,color,size,alpha,label,file_type,shp,gdb,layer,display,Unnamed: 11,Unnamed: 12,Unnamed: 13
20,Proposed Pipeline,line,orange,4.0,1.0,Label,shp,\\ppeng.com\pzdata\clients\Arvin-Edison WSD-12...,NaN,NaN,1.0,NaN,NaN,NaN
21,Frick Unit Service Area,filled_polygon,red,4.0,0.7,Name,shp,\\ppeng.com\pzdata\clients\Arvin-Edison WSD-12...,NaN,NaN,1.0,NaN,NaN,NaN
22,Proposed Turnout,point,yellow,30.0,1.0,Label,shp,\\ppeng.com\pzdata\clients\Arvin-Edison WSD-12...,NaN,NaN,1.0,NaN,NaN,NaN


In [2]:
# row = config.iloc[-1]
# gpd.read_file(Path(row["shp"]))

In [3]:
def get_layer(row):
    try:
        logger.info(f"Loading layer {row['Name']}")
        if row["display"] == True:
            if row["file_type"] == "shp":
                gdf = gpd.read_file(Path(row["shp"]))
            elif row["file_type"] == "gdb":
                gdb = Path(row["gdb"])
                gdf = gpd.read_file(gdb, layer=row["layer"])
            else:
                return None

            gdf["color"] = row["color"]
            # gdf["label"] = gdf[row["label"]].astype(str)
            # print(gdf.columns)
            gdf["label"] = gdf[row["label"]].astype(str).replace("nan", "")

            gdf["layer"] = row["Name"]
            gdf["size"] = row["size"]
            # print(row['Name'])
            # print(gdf.crs)

            return gdf[
                ["color", "label", "layer", "size", "geometry"]
            ]  # .to_crs(epsg=4326)
        else:
            logger.info(f"Layer {row['Name']} is not displayed")
            # print(row['Name'])
            return None
    except Exception as e:
        logger.error(f"Layer {row['Name']} failed to load due to {e}")
        return None



# Create gdb
gdfs = {y["Name"]: get_layer(y) for i, y in config.iterrows()}
# gdfs = {y['Name']:y for i,y in config.iterrows()}

# gdfs['Proposed Pipeline']

2024-01-15 17:11:54.279 | INFO     | __main__:get_layer:4 - Loading layer Proposed Pipeline
2024-01-15 17:11:54.280 | INFO     | __main__:get_layer:4 - Loading layer APNs
2024-01-15 17:11:54.281 | INFO     | __main__:get_layer:4 - Loading layer District Boundary
2024-01-15 17:11:54.518 | INFO     | __main__:get_layer:4 - Loading layer AEWSD North Canal
2024-01-15 17:11:54.563 | INFO     | __main__:get_layer:4 - Loading layer FFPPP Discharge Pipeline
2024-01-15 17:11:54.606 | INFO     | __main__:get_layer:4 - Loading layer Frick Unit Service Area
2024-01-15 17:11:54.607 | ERROR    | __main__:get_layer:27 - Layer Frick Unit Service Area is not displayed
2024-01-15 17:11:54.608 | INFO     | __main__:get_layer:4 - Loading layer Proposed Turnout
2024-01-15 17:11:54.608 | ERROR    | __main__:get_layer:27 - Layer Proposed Turnout is not displayed
2024-01-15 17:11:54.609 | INFO     | __main__:get_layer:4 - Loading layer DiGiorgio Pipeline
2024-01-15 17:11:54.659 | INFO     | __main__:get_layer

In [6]:
epsg = 4326
# epsg = 26745
# epsg = 2229
# epsg = 6424
import arrow

gdf = pd.concat([gdf.to_crs(epsg=epsg) for gdf in gdfs.values() if gdf is not None])

gdf.to_parquet(data_path / f"gdf-{arrow.now().format('YYYY-MM-DD')}.parquet")